In [1]:
import random
import numpy as np

class GeneticAlgorithmKnapsack:
    def __init__(self, v, w, K, N=100, s=0.5, c=0.4, m=0.1, maxI=1000, alpha=1):
        self.v = np.array(v)  # Valores
        self.w = np.array(w)  # Pesos
        self.K = K            # Capacidad de la mochila
        self.N = N            # Tamaño de la población
        self.s = s            # Porcentaje de selección
        self.c = c            # Porcentaje de cruce
        self.m = m            # Porcentaje de mutación
        self.maxI = maxI      # Número máximo de iteraciones
        self.alpha = alpha    # Factor de penalización
        self.n_items = len(v) # Número de objetos
        self.population = []  # Población actual
        self.best_solution = None  # Mejor solución encontrada
        self.best_fitness = -np.inf

    def initialize_population(self):
        """Genera la población inicial."""
        self.population = []
        for _ in range(self.N):
            individual = np.random.randint(2, size=self.n_items)
            self.population.append(individual)

    def fitness(self, individual):
        """Calcula el fitness de un individuo."""
        total_value = np.sum(individual * self.v)
        total_weight = np.sum(individual * self.w)
        if total_weight <= self.K:
            return total_value
        else:
            return total_value - self.alpha * (total_weight - self.K)

    def evaluate_population(self):
        """Evalúa toda la población y actualiza el mejor individuo."""
        fitness_values = []
        for individual in self.population:
            fit = self.fitness(individual)
            fitness_values.append(fit)
            if fit > self.best_fitness:
                self.best_fitness = fit
                self.best_solution = individual.copy()
        return fitness_values

    def selection(self, fitness_values):
        """Selecciona individuos para la siguiente generación."""
        total_fitness = sum(fitness_values)
        selection_probs = [f / total_fitness for f in fitness_values]
        num_selected = int(self.s * self.N)
        selected_indices = np.random.choice(range(self.N), size=num_selected, p=selection_probs)
        selected = [self.population[i] for i in selected_indices]
        return selected

    def crossover(self, parents):
        """Aplica el operador de cruce para generar nuevos individuos."""
        num_offspring = int(self.c * self.N)
        offspring = []
        while len(offspring) < num_offspring:
            parent1, parent2 = random.sample(parents, 2)
            crossover_point = random.randint(1, self.n_items - 1)
            child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            offspring.extend([child1, child2])
        return offspring[:num_offspring]

    def mutation(self, individuals):
        """Aplica el operador de mutación a los individuos."""
        num_mutations = int(self.m * self.N)
        mutated = []
        for _ in range(num_mutations):
            individual = random.choice(individuals).copy()
            mutation_point = random.randint(0, self.n_items - 1)
            individual[mutation_point] = 1 - individual[mutation_point]  # Flip bit
            mutated.append(individual)
        return mutated

    def create_new_population(self, selected, offspring, mutated):
        """Crea la nueva población combinando los individuos."""
        self.population = selected + offspring + mutated
        # Si la población es mayor que N, recortar
        if len(self.population) > self.N:
            self.population = self.population[:self.N]

    def run(self):
        """Ejecuta el algoritmo genético."""
        self.initialize_population()
        for iteration in range(self.maxI):
            fitness_values = self.evaluate_population()
            selected = self.selection(fitness_values)
            offspring = self.crossover(selected)
            mutated = self.mutation(self.population)
            self.create_new_population(selected, offspring, mutated)
            # Opcional: Mostrar progreso
            if (iteration + 1) % 100 == 0 or iteration == 0:
                print(f"Iteración {iteration + 1}, Mejor fitness: {self.best_fitness}")
        return self.get_result()

    def get_result(self):
        """Devuelve la mejor solución encontrada."""
        selected_items = [i for i in range(self.n_items) if self.best_solution[i] == 1]
        total_value = np.sum(self.best_solution * self.v)
        total_weight = np.sum(self.best_solution * self.w)
        return {
            'selected_items': selected_items,
            'total_value': total_value,
            'total_weight': total_weight
        }


In [2]:
# Valores y pesos de los objetos
v = [60, 100, 120]
w = [10, 20, 30]
K = 50  # Capacidad de la mochila

# Parámetros del algoritmo genético
N = 100       # Tamaño de la población
s = 0.5       # Porcentaje de selección
c = 0.4       # Porcentaje de cruce
m = 0.1       # Porcentaje de mutación
maxI = 500    # Número máximo de iteraciones
alpha = 10    # Factor de penalización


In [3]:
# Crear una instancia del algoritmo genético
ga_knapsack = GeneticAlgorithmKnapsack(v, w, K, N, s, c, m, maxI, alpha)

# Ejecutar el algoritmo
result = ga_knapsack.run()

# Mostrar los resultados
print("\nMejor solución encontrada:")
print(f"Objetos seleccionados: {result['selected_items']}")
print(f"Valor total: {result['total_value']}")
print(f"Peso total: {result['total_weight']}")

Iteración 1, Mejor fitness: 220
Iteración 100, Mejor fitness: 220
Iteración 200, Mejor fitness: 220
Iteración 300, Mejor fitness: 220
Iteración 400, Mejor fitness: 220
Iteración 500, Mejor fitness: 220

Mejor solución encontrada:
Objetos seleccionados: [1, 2]
Valor total: 220
Peso total: 50
